# Named Tensor Notation

## Introduction

**This is a translation of the [Named Tensor Notation (Chiang, Rush, Barak 2021)](https://namedtensor.github.io/) example from the [funsor](https://funsor.pyro.ai/) library to `effectful`. Much of the expository text is taken directly from the [original](https://github.com/pyro-ppl/funsor/blob/master/tutorials/named_tensor_notation_i.ipynb).**

The mathematical notation with *named axes* introduced in [Named Tensor Notation (Chiang, Rush, Barak 2021)](https://namedtensor.github.io/) improves the readability of mathematical formulas involving multidimensional arrays. This includes tensor operations such as elementwise operations, reductions, contractions, renaming, indexing, and broadcasting. Part 1 covers examples from [2 Informal Overview](https://namedtensor.github.io/#sec:overview), [3.4.2 Advanced Indexing](https://namedtensor.github.io/#sec:examples), and [5 Formal Definitions](https://namedtensor.github.io/#sec:definitions).

In [ ]:
%reload_ext autoreload
%autoreload 2

import functools
from typing import Annotated, TypeVar

import torch
from torch import tensor

from effectful.handlers.torch import sizesof, to_tensor
from effectful.ops.semantics import evaluate, fvsof, handler
from effectful.ops.syntax import Scoped, defop
from effectful.ops.types import Operation

S1, S2, S3 = TypeVar("S1"), TypeVar("S2"), TypeVar("S3")


def subst(term, substs):
    with handler(
        {k: functools.partial(lambda vv: vv, v) for (k, v) in substs.items()},
    ):
        return evaluate(term)


def reduce(indexes, indexed_tensor, reducer):
    """Reduce an indexed tensor along one or more named dimensions.

    Args:
    - indexes: Names of dimensions to reduce.
    - indexed_tensor: The tensor to reduce.
    - reducer: A reduction function like `torch.sum`. Must take `tensor`, `dim`, and `keepdim` arguments.

    Returns: A new indexed tensor with the specified dimensions reduced.

    Example:
    >>> width, height = defop(torch.Tensor, name='width'), defop(torch.Tensor, name='height')
    >>> t = indexed(torch.ones(2, 3))[width(), height()]
    >>> reduce([width], t, "sum")
    indexed(tensor([2., 2., 2.]))[height()]
    """
    fvars = fvsof(indexed_tensor)
    indexes = [i for i in indexes if i in fvars]

    # convert indexed dimensions to positional and flatten all new positional dims
    t = to_tensor(indexed_tensor, *indexes)
    t_flat = torch.flatten(t, 0, len(indexes) - 1)

    # reduce dim 0 into the first index of dim 0, then return reduction
    return reducer(t_flat, 0, keepdim=True)[0]


@functools.cache
def name_to_symbol(name: str) -> Operation[[], torch.Tensor]:
    """
    Create a persistent Operation symbol to use as a dimension name.
    We memoize this function because `defop` returns a fresh symbol each time it's called,
    and we want to be able to safely run notebook cells out of order.
    """
    return defop(torch.Tensor, name=name)

## Named Tensors

Each tensor axis is given a name:

$$
\begin{aligned}
  A &\in \mathbb{R}^{\mathsf{\vphantom{fg}height}[3] \times \mathsf{\vphantom{fg}width}[3]} = \mathbb{R}^{\mathsf{\vphantom{fg}width}[3] \times \mathsf{\vphantom{fg}height}[3]} \\
  A &= \mathsf{\vphantom{fg}height}
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
    3 & 1 & 4 \\
    1 & 5 & 9 \\
    2 & 6 & 5
  \end{bmatrix}\end{array} =
  \mathsf{\vphantom{fg}width}
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\\begin{bmatrix}
    3 & 1 & 2 \\
    1 & 5 & 6 \\
    4 & 9 & 5
  \end{bmatrix}\end{array}.
\end{aligned}
$$

In [168]:
height, width = name_to_symbol("height"), name_to_symbol("width")
t = tensor([[3, 1, 4], [1, 5, 9], [2, 6, 5]])
A = t[height(), width()]
A

tensor([[3, 1, 4],
        [1, 5, 9],
        [2, 6, 5]])[height(), width()]

Access elements of $A$ using named indices:

$$
A_{\mathsf{\vphantom{fg}height}(1), \mathsf{\vphantom{fg}width}(3)} = A_{\mathsf{\vphantom{fg}width}(3), \mathsf{\vphantom{fg}height}(1)} = 4
$$

In [169]:
subst(A, {height: 0, width: 2})

tensor(4)

Partial indexing:

$$
\begin{aligned}
A_{\mathsf{\vphantom{fg}height}(1)} &= \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3 & 1 & 4
\end{bmatrix}\end{array}
&
A_{\mathsf{\vphantom{fg}width}(3)} &= \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\
\begin{bmatrix}
  4 & 9 & 5
\end{bmatrix}\end{array}.
\end{aligned}
$$

In [170]:
subst(A, {height: 0})

tensor([3, 1, 4])[width()]

In [171]:
subst(A, {width: 2})

tensor([4, 9, 5])[height()]

## Named Tensor Operations

### Elementwise Operations and Broadcasting

Elementwise operations:

$$
\frac1{1+\exp(-A)} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  \frac 1{1+\exp(-3)} & \frac 1{1+\exp(-1)} & \frac 1{1+\exp(-4)} \\[1ex]
  \frac 1{1+\exp(-1)} & \frac 1{1+\exp(-5)} & \frac 1{1+\exp(-9)} \\[1ex]
  \frac 1{1+\exp(-2)} & \frac 1{1+\exp(-6)} & \frac 1{1+\exp(-5)}
\end{bmatrix}\end{array}.
$$

In [172]:
1 / (1 + (-A).exp())

tensor([[0.9526, 0.7311, 0.9820],
        [0.7311, 0.9933, 0.9999],
        [0.8808, 0.9975, 0.9933]])[height(), width()]

Tensors with different shapes are automatically broadcasted against each other before an operation is applied. Let

$$
\begin{aligned}
  x &\in \mathbb{R}^{\mathsf{\vphantom{fg}height}[3]} & y &\in \mathbb{R}^{\mathsf{\vphantom{fg}width}[3]} \\
  x &= \mathsf{\vphantom{fg}height}
  \begin{array}[b]{@{}c@{}}\\
  \begin{bmatrix}
    2 \\ 7 \\ 1
  \end{bmatrix}\end{array} & 
  y &= 
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
    1 & 4 & 1
  \end{bmatrix}\end{array}.
\end{aligned}
$$

In [173]:
x = tensor([2, 7, 1])[height()]
y = tensor([1, 4, 1])[width()]

Binary addition operation:

$$
\begin{aligned}
A + x &= \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3+2 & 1+2 & 4+2 \\
  1+7 & 5+7 & 9+7 \\
  2+1 & 6+1 & 5+1
\end{bmatrix}\end{array} &
A + y &= \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3+1 & 1+4 & 4+1 \\
  1+1 & 5+4 & 9+1 \\
  2+1 & 6+4 & 5+1
\end{bmatrix}\end{array}.
\end{aligned}
$$

In [174]:
A + x

tensor([[ 5,  3,  6],
        [ 8, 12, 16],
        [ 3,  7,  6]])[height(), width()]

In [175]:
A + y

tensor([[ 4,  5,  5],
        [ 2,  9, 10],
        [ 3, 10,  6]])[height(), width()]

Binary multiplication operation:

$$
A \odot x = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3\cdot2 & 1\cdot2 & 4\cdot2 \\
  1\cdot7 & 5\cdot7 & 9\cdot7 \\
  2\cdot1 & 6\cdot1 & 5\cdot1
\end{bmatrix}\end{array}
$$

In [176]:
A * x

tensor([[ 6,  2,  8],
        [ 7, 35, 63],
        [ 2,  6,  5]])[height(), width()]

Binary maximum operation:

$$
\max(A, y) = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  \max(3, 1) & \max(1, 4) & \max(4, 1) \\
  \max(1, 1) & \max(5, 4) & \max(9, 1) \\
  \max(2, 1) & \max(6, 4) & \max(5, 1)
\end{bmatrix}\end{array}.
$$

In [177]:
torch.max(A, y)

tensor([[3, 4, 4],
        [1, 5, 9],
        [2, 6, 5]])[height(), width()]

### Reductions

Named axes can be reduced over by calling the `.reduce` method and specifying the [reduction operator](https://en.wikipedia.org/wiki/Reduction_Operator) and names of reduced axes. Note that reduction is defined only for operators that are associative and commutative.

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \sum_i A_{\mathsf{\vphantom{fg}height}(i)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3+1+2 & 1+5+6 & 4+9+5
\end{bmatrix}\end{array}.
$$

In [178]:
reduce([height], A, torch.sum)

tensor([ 6, 12, 18])[width()]

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}width}}} A = \sum_j A_{\mathsf{\vphantom{fg}width}(j)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\
\begin{bmatrix}
  3+1+4 & 1+5+9 & 2+6+5
\end{bmatrix}\end{array}.
$$

In [179]:
reduce([width], A, torch.sum)

tensor([ 8, 15, 13])[height()]

Reduction over multiple axes:

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}height}\\
 \mathsf{\vphantom{fg}width}}} A = \sum_i \sum_j A_{\mathsf{\vphantom{fg}height}(i),\mathsf{\vphantom{fg}width}(j)} = 3+1+4+1+5+9+2+6+5.
 $$

In [180]:
reduce([height, width], A, torch.sum)

tensor(36)

Multiplication reduction:

$$
\prod\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \prod_i A_{\mathsf{\vphantom{fg}height}(i)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3\cdot1\cdot2 & 1\cdot5\cdot6 & 4\cdot9\cdot5
\end{bmatrix}\end{array}.
$$

In [181]:
reduce([height], A, torch.prod)

tensor([  6,  30, 180])[width()]

Max reduction:

$$
\max\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \max \{A_{\mathsf{\vphantom{fg}height}(i)} \mid 1 \leq i \leq n\} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  \max(3, 1, 2) & \max(1, 5, 6) & \max(4, 9, 5)
\end{bmatrix}\end{array}.
$$

In [182]:
reduce([height], A, torch.amax)

tensor([3, 6, 9])[width()]

### Contraction

Contraction operation can be written as elementwise multiplication followed by summation over an axis:

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} y = \sum_j A_{\mathsf{\vphantom{fg}width}(j)} \, y_{\mathsf{\vphantom{fg}width}(j)} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\\\begin{bmatrix}
  3\cdot 1 + 1\cdot 4 + 4\cdot 1 \\
  1\cdot 1 + 5\cdot 4 + 9\cdot 1 \\
  2\cdot 1 + 6\cdot 4 + 5\cdot 1
\end{bmatrix}\end{array}.
$$

In [183]:
reduce([width], A * y, torch.sum)

tensor([11, 30, 31])[height()]

Some other operations from linear algebra:

$$
x \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}}}{\vphantom{fg}\odot}} x = \sum_i x_{\mathsf{\vphantom{fg}height}(i)} \, x_{\mathsf{\vphantom{fg}height}(i)} \qquad \text{inner product}
$$

In [184]:
reduce([height], x * x, torch.sum)

tensor(54)

$$
[x \odot y]_{\mathsf{\vphantom{fg}height}(i), \mathsf{\vphantom{fg}width}(j)} = x_{\mathsf{\vphantom{fg}height}(i)} \, y_{\mathsf{\vphantom{fg}width}(j)} \qquad \text{outer product}
$$

In [185]:
x * y

tensor([[ 2,  8,  2],
        [ 7, 28,  7],
        [ 1,  4,  1]])[height(), width()]

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} y = \sum_i A_{\mathsf{\vphantom{fg}width}(i)} \, y_{\mathsf{\vphantom{fg}width}(i)} \qquad \text{matrix-vector product}
$$

In [186]:
reduce([width], A * y, torch.sum)

tensor([11, 30, 31])[height()]

$$
x \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}}}{\vphantom{fg}\odot}} A = \sum_i x_{\mathsf{\vphantom{fg}height}(i)} \, A_{\mathsf{\vphantom{fg}height}(i)} \qquad \text{vector-matrix product} \\
$$

In [187]:
reduce([height], x * A, torch.sum)

tensor([15, 43, 76])[width()]

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} B = \sum_i A_{\mathsf{\vphantom{fg}width}(i)} \odot B_{\mathsf{\vphantom{fg}width}(i)} \qquad \text{matrix-matrix product}~(B \in \mathbb{R}^{\mathsf{\vphantom{fg}width}\times \mathsf{\vphantom{fg}width2}})
$$

In [188]:
width2 = name_to_symbol("width2")
B = tensor([[3, 2, 5], [5, 4, 0], [8, 3, 6]])[width(), width2()]
reduce([width], A * B, torch.sum)

tensor([[ 46,  22,  39],
        [100,  49,  59],
        [ 76,  43,  40]])[height(), width2()]

Contraction can be generalized to other binary and reduction operations:

$$
\max_{\mathsf{\vphantom{fg}width}} (A + y) = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\\\begin{bmatrix}
  \max(3+1, 1+4, 4+1) \\
  \max(1+1, 5+4, 9+1) \\
  \max(2+1, 6+4, 5+1)
\end{bmatrix}\end{array}.
$$

In [189]:
reduce([width], A + y, torch.amax)

tensor([ 5, 10, 10])[height()]

### Renaming and Reshaping

Renaming named dimensions is simple:

$$
A_{\mathsf{\vphantom{fg}height}\rightarrow\mathsf{\vphantom{fg}height2}} = \mathsf{\vphantom{fg}height2}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}
\\\begin{bmatrix}
  3 & 1 & 4 \\
  1 & 5 & 9 \\
  2 & 6 & 5 \\
\end{bmatrix}\end{array}.
$$

In [190]:
height2 = name_to_symbol("height2")
A2 = subst(A, {height: height2()})
print(A2)

tensor([[3, 1, 4],
        [1, 5, 9],
        [2, 6, 5]])[height2(), width()]


$$
A_{(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width})\rightarrow\mathsf{\vphantom{fg}layer}} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}layer}\\
\begin{bmatrix}
    3 & 1 & 4 & 1 & 5 & 9 & 2 & 6 & 5
\end{bmatrix}\end{array}
$$

In [191]:
layer = name_to_symbol("layer")
A_layer = subst(
    A, {height: layer() // torch.tensor(3), width: layer() % torch.tensor(3)}
)
A_layer_2 = subst(A_layer, {layer: torch.tensor(2)})
print(A_layer_2)

tensor(4)


$$
A_{\mathsf{\vphantom{fg}layer}\rightarrow(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width})} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}
\\\begin{bmatrix}
  3 & 1 & 4 \\
  1 & 5 & 9 \\
  2 & 6 & 5 \\
\end{bmatrix}\end{array}.
$$

In [192]:
A_layer_hw = subst(A_layer, {layer: height2() * 3 + width() % 3})
print(A_layer_hw)

torch_getitem(tensor([[3, 1, 4],
        [1, 5, 9],
        [2, 6, 5]]), ['floor_divide(add(mul(height2(), 3), fmod(width(), 3)), tensor(3))', 'fmod(add(mul(height2(), 3), fmod(width(), 3)), tensor(3))'])


## Advanced Indexing

All of advanced indexing can be achieved through name substitutions.

$$
\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{index}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}ax}[n]} \times [n] \rightarrow \mathbb{R}\\
\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{index}}}(A, i) = A_{\mathsf{\vphantom{fg}ax}(i)}.
$$

$$
\begin{aligned}
  E &\in \mathbb{R}^{\mathsf{\vphantom{fg}vocab}[n] \times \mathsf{\vphantom{fg}emb}} \\
  i &\in [n] \\
  I &\in [n]^{\mathsf{\vphantom{fg}seq}} \\
  P &\in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}vocab}[n]}
\end{aligned}
$$

Partial indexing $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(E,i)$:

In [193]:
vocab, emb = name_to_symbol("vocab"), name_to_symbol("emb")
E = torch.tensor([[2, 1, 5], [3, 4, 2], [1, 3, 7], [1, 4, 3], [5, 9, 2]])[
    vocab(), emb()
]
subst(E, {vocab: 2})

tensor([1, 3, 7])[emb()]

Integer array indexing $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(E,I)$:

In [194]:
seq = name_to_symbol("seq")
I0 = torch.tensor([3, 2, 4, 0])[seq()]
subst(E, {vocab: I0})

tensor([[1, 4, 3],
        [1, 3, 7],
        [5, 9, 2],
        [2, 1, 5]])[seq(), emb()]

Gather operation $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(P,I)$:

In [195]:
P = torch.tensor(
    [[6, 2, 4, 2], [8, 2, 1, 3], [5, 5, 7, 0], [1, 3, 8, 2], [5, 9, 2, 3]]
)[vocab(), seq()]

subst(P, {vocab: I0})

tensor([1, 5, 2, 2])[seq()]

Indexing with two integer arrays:

$$
\begin{aligned}
  |\mathsf{\vphantom{fg}seq}| &= m \\
  I_1 &= [m]^\mathsf{\vphantom{fg}subseq}\\
  I_2 &= [n]^\mathsf{\vphantom{fg}subseq}\\
  S &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(\mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\mathrm{index}}}(P, I_1), I_2) \in \mathbb{R}^{\mathsf{\vphantom{fg}subseq}} \\
  S_{\mathsf{\vphantom{fg}subseq}(i)} &= P_{\mathsf{\vphantom{fg}seq}(I_{\mathsf{\vphantom{fg}subseq}(i)}), \mathsf{\vphantom{fg}vocab}(I_{\mathsf{\vphantom{fg}subseq}(i)})}.
\end{aligned}
$$

In [196]:
subseq = name_to_symbol("subseq")
I1 = torch.tensor([1, 2, 0])[subseq()]
I2 = torch.tensor([3, 0, 4])[subseq()]

subst(P, {seq: I1, vocab: I2})

tensor([3, 4, 5])[subseq()]

## Constructing Neural Networks

### Feedforward

\begin{aligned}
  X^0 &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}} \\
  X^1 &= \sigma(W^1 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}input}}}{\vphantom{fg}\odot}} X^0 + b^1) & W^1 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_1 \times \mathsf{\vphantom{fg}input}} & b^1 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_1} \\
  X^2 &= \sigma(W^2 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}_1}}{\vphantom{fg}\odot}} X^1 + b^2) & W^2 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_2 \times \mathsf{\vphantom{fg}hidden}_1} & b^2 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_2} \\
  X^3 &= \sigma(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}_2}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}out}\times \mathsf{\vphantom{fg}hidden}_2} & b^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}out}}
\end{aligned}

$$
\begin{aligned}
x &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer}[n_0]} \\
W^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer^2}[n_l] \times \mathsf{\vphantom{fg}layer}[n_{l-1}]} \\
  b^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer^2}[n_l]} \\
  \text{FullConn}^l(x) &= \sigma\left(W^l \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} x + b^l\right)_{\mathsf{\vphantom{fg}layer^2}\rightarrow\mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

In [197]:
@defop
def FullConn(
    x: Annotated[torch.Tensor, Scoped[S1]],
    W: Annotated[torch.Tensor, Scoped[S1]],
    b: Annotated[torch.Tensor, Scoped[S1]],
    layer: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    return reduce([layer], torch.sigmoid(torch.mul(W, x)), torch.sum) + b

In [198]:
input_size = 100
output_size = 32
input_, output = name_to_symbol("input"), name_to_symbol("output")

W = torch.randn(input_size, output_size)[input_(), output()]
b = torch.randn(output_size)[output()]
X = torch.randn(input_size)[input_()]

FullConn(X, W, b, input_)

tensor([49.6214, 49.1620, 46.7234, 48.5531, 49.5717, 49.0428, 49.3330, 50.5993,
        50.1553, 49.8182, 50.9401, 49.1957, 49.4627, 51.3990, 50.4449, 48.4399,
        49.1624, 53.8431, 49.3302, 47.4099, 49.5286, 50.8544, 50.5048, 54.8563,
        48.1085, 51.1001, 49.7631, 52.9175, 48.7131, 50.6819, 49.1625, 50.3209])[output()]

### Recurrent

$$
\begin{aligned}
x^{t} &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}} & t &= 1, \ldots, n \\
W^{\text{h}} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}hidden}^\prime} & |\mathsf{\vphantom{fg}hidden}| &= |\mathsf{\vphantom{fg}hidden}^\prime| \\
W^{\text{i}} &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}\times \mathsf{\vphantom{fg}hidden}^\prime} \\
b &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}^\prime} \\
h^{0} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
h^{t} &= \sigma\left( W^{\text{h}} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} h^{t-1} + W^{\text{i}} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}input}}}{\vphantom{fg}\odot}} x^{t} + b \right)_{\mathsf{\vphantom{fg}hidden}^\prime\rightarrow\mathsf{\vphantom{fg}hidden}} & t &= 1, \ldots, n
\end{aligned}
$$

In [199]:
@defop
def RNN(
    x: Annotated[torch.Tensor, Scoped[S1]],
    Wh: Annotated[torch.Tensor, Scoped[S1]],
    Wi: Annotated[torch.Tensor, Scoped[S1]],
    b: Annotated[torch.Tensor, Scoped[S1]],
    h: Annotated[torch.Tensor, Scoped[S1]],
    hidden: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    layer: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    return torch.sigmoid(
        reduce([hidden], Wh * h, torch.sum) + reduce([layer], Wi * x, torch.sum) + b
    )

In [200]:
input_size = 100
hidden_size = 32
input_, hidden, hidden2 = map(name_to_symbol, ("input", "hidden", "hidden2"))

Wh = torch.randn(hidden_size, hidden_size)[hidden(), hidden2()]
Wi = torch.randn(input_size, hidden_size)[input_(), hidden2()]
b = torch.randn(hidden_size)[hidden2()]
h = torch.randn(hidden_size)[hidden()]
x = torch.randn(input_size)[input_()]

RNN(x, Wh, Wi, b, h, hidden, input_)

tensor([5.8642e-13, 9.8138e-01, 9.9968e-01, 9.9999e-01, 3.2927e-05, 9.9999e-01,
        6.9062e-04, 5.1937e-02, 1.2397e-05, 9.9914e-01, 7.7800e-03, 1.0000e+00,
        5.7578e-04, 2.0352e-02, 1.5684e-02, 8.6293e-05, 2.7781e-01, 4.2824e-02,
        9.9636e-01, 9.9777e-01, 2.8032e-01, 1.9554e-02, 9.9816e-01, 2.5286e-02,
        1.9089e-04, 2.7259e-09, 9.9995e-01, 9.3336e-01, 6.4968e-07, 3.3902e-05,
        4.0663e-01, 1.2525e-01])[hidden2()]

### Attention

In [201]:
@defop
def Softmax(
    x: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    ax2: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    x = subst(x, {ax: ax2()})
    y = x - reduce([ax2], x, torch.logsumexp)
    return y.exp()

\begin{aligned}
  \text{Attention} \colon \mathbb{R}^{\mathsf{\vphantom{fg}key}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times\mathsf{\vphantom{fg}key}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times\mathsf{\vphantom{fg}val}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}val}} \\
\text{Attention}(Q, K, V, M) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\mathrm{softmax}}} \left( \frac{Q \mathbin{\underset{\substack{\mathsf{\vphantom{fg}key}}}{\vphantom{fg}\odot}} K}{\sqrt{|\mathsf{\vphantom{fg}key}|}} + M \right) \mathbin{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\odot}} V.
\end{aligned}

In [202]:
@defop
def Attention(
    Q: Annotated[torch.Tensor, Scoped[S1]],
    K: Annotated[torch.Tensor, Scoped[S1]],
    V: Annotated[torch.Tensor, Scoped[S1]],
    M: Annotated[torch.Tensor, Scoped[S1]],
    key: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    seq: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    seq2: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    x = reduce([key], Q * K, torch.sum) / sizesof(Q)[key] + M
    return reduce([seq], Softmax(x, seq, seq2) * V, torch.sum)

In [203]:
key_size = 10
val_size = 5
seq_size = 3

key, val, seq, seq2 = map(name_to_symbol, ("key", "val", "seq", "seq2"))
Q = torch.randn(key_size)[key()]
K = torch.randn(key_size, seq_size)[key(), seq()]
V = torch.randn(seq_size, val_size)[seq(), val()]
M = torch.randn(seq_size)[seq()]

Attention(Q, K, V, M, key, seq, seq2)

tensor([[0.1311, 0.2201, 0.2817, 0.1219, 0.1279],
        [0.7969, 1.3385, 1.7125, 0.7410, 0.7779],
        [0.0980, 0.1647, 0.2107, 0.0912, 0.0957]])[seq2(), val()]

### Convolution

\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n-|\mathsf{\vphantom{fg}kernel}|+1], \mathsf{\vphantom{fg}kernel}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} X &= Y,\ \text{where} \\
  Y_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}kernel}(j)} &= X_{\mathsf{\vphantom{fg}seq}(i+j - 1)}.
\end{aligned}

In [ ]:
@defop
def Unroll(
    x: Annotated[torch.Tensor, Scoped[S1]],
    seq: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    k: int,
    kernel: Operation[[], torch.Tensor],
    seq2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    return to_tensor(x, seq).unfold(0, k, 1)[seq2(), kernel()]

\begin{aligned}
\text{Conv1d} \colon \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n^\prime]} \\
\text{Conv1d}(X; W, b) &= W \mathbin{\underset{\substack{\mathsf{\vphantom{fg}chans}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\odot}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} X + b
\end{aligned}

\begin{aligned}
W &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}kernel}} \\
b &\in \mathbb{R}\\
\end{aligned}

In [205]:
@defop
def Conv1d(
    X: Annotated[torch.Tensor, Scoped[S1 | S2]],
    W: Annotated[torch.Tensor, Scoped[S2]],
    b: torch.Tensor,
    chans: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    k: int,
    kernel: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    seq: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    seq2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    y = W * Unroll(X, seq, k, kernel, seq2)
    return reduce([chans, kernel], y, torch.sum) + b

In [206]:
chans_size = 3
seq_size = 10
kernel_size = 3

chans, kernel, seq, seq2 = map(name_to_symbol, ("chans", "kernel", "seq", "seq2"))

X = torch.randn(chans_size, seq_size)[chans(), seq()]
W = torch.randn(chans_size, kernel_size)[chans(), kernel()]
b = torch.randn(tuple())

Conv1d(X, W, b, chans, 3, kernel, seq, seq2)

tensor([ 1.3124,  0.1102, -6.1440, -1.6563, -5.7074,  3.8315, -2.5586, -3.6780])[seq2()]

$$
\begin{aligned}
  \text{Conv2d} \colon \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}height}[h] \times \mathsf{\vphantom{fg}width}[w]}
  &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}height}[h2] \times \mathsf{\vphantom{fg}width}[w2]} \\
  \text{Conv2d}(X; W, b) &= W \mathbin{\underset{\substack{\mathsf{\vphantom{fg}chans}\\ \mathsf{\vphantom{fg}kh}, \mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\odot}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}height}\\ \mathsf{\vphantom{fg}kh}}}{\vphantom{fg}\mathrm{unroll}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}width}\\\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{unroll}}} X + b\end{aligned}
$$

$$
\begin{aligned}
W &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}kh}\times \mathsf{\vphantom{fg}kw}} \\
b &\in \mathbb{R}.
\end{aligned}
$$

In [207]:
@defop
def Conv2d(
    X: Annotated[torch.Tensor, Scoped[S1 | S2]],
    W: Annotated[torch.Tensor, Scoped[S2]],
    b: torch.Tensor,
    chans: Annotated[Operation[[], torch.Tensor], Scoped[S2]],
    kh_size: int,
    kh: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    height: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    height2: Operation[[], torch.Tensor],
    kw_size: int,
    kw: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    width: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    width2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    y = W * Unroll(Unroll(X, width, kw_size, kw, width2), height, kh_size, kh, height2)
    return reduce([chans, kh, kw], y, torch.sum) + b

In [208]:
chans_size = 3
kh_size = 3
kw_size = 4
height_size = 10
width_size = 8

chans, kh, kw, height, width, height2, width2 = map(
    name_to_symbol, ("chans", "kh", "kw", "height", "width", "height2", "width2")
)

X = torch.randn(chans_size, height_size, width_size)[chans(), height(), width()]
W = torch.randn(chans_size, kh_size, kw_size)[chans(), kh(), kw()]
b = torch.randn(tuple())

Conv2d(X, W, b, chans, kh_size, kh, height, height2, kw_size, kw, width, width2)

tensor([[ 3.1259, -2.5779, -8.5917, -5.2500,  0.9331, -1.0349, -2.5460, -7.8026],
        [-2.4204,  2.6947, -4.3422, -4.1924, -1.6958, -4.8411,  6.3812,  3.1479],
        [ 8.0389, -3.5227,  8.6371, -7.9508,  3.3021, -4.4417,  4.8447, -7.3063],
        [ 7.1624, -0.9561,  6.7281, -1.8662,  1.8822,  0.0897,  1.0368, -4.5366],
        [ 3.3856,  0.3371, -4.8247, -2.9813, 10.4098, -1.9862, -4.5428, -2.5016]])[width2(), height2()]

### Max Pooling

$$
\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n/|\mathsf{\vphantom{fg}kernel}|],\mathsf{\vphantom{fg}kernel}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X &= Y,\ \text{where} \\
  Y_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}kernel}(j)} &= X_{\mathsf{\vphantom{fg}seq}((i-1) \cdot |\mathsf{\vphantom{fg}kernel}| + j)}.
\end{aligned}
$$

In [ ]:
@defop
def Pool(
    x: Annotated[torch.Tensor, Scoped[S1]],
    seq: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    k: int,
    kernel: Operation[[], torch.Tensor],
    seq2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    xp = to_tensor(x, seq)
    return xp.reshape((xp.shape[0] // k, k) + xp.shape[1:])[seq2(), kernel()]

In [210]:
seq_size = 10
seq, seq2, kernel = map(name_to_symbol, ("seq", "seq2", "kernel"))

X = torch.randn(seq_size)[seq()]
Y = Pool(X, seq, 2, kernel, seq2)
Y

tensor([[ 0.1942,  2.0733],
        [ 0.1388,  0.7844],
        [-0.8563, -0.3301],
        [ 1.0066, -0.4537],
        [ 1.3682, -0.9057]])[seq2(), kernel()]

$$
\begin{aligned}
\text{MaxPool1d}_{k} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n/k]} \\
\text{MaxPool1d}_{k}(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{max}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X \\
|\mathsf{\vphantom{fg}kernel}| &= k \\
\text{MaxPool2d}_{kh,kw} \colon \mathbb{R}^{\mathsf{\vphantom{fg}height}[h] \times \mathsf{\vphantom{fg}width}[w]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}height}[h/kh] \times \mathsf{\vphantom{fg}width}[w/kw]} \\
\text{MaxPool2d}_{kh,kw}(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}kh},\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{max}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}kh}}}{\vphantom{fg}\mathrm{pool}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}width},\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{pool}}} X \\
|\mathsf{\vphantom{fg}kh}| &= kh \\
|\mathsf{\vphantom{fg}kw}| &= kw.
\end{aligned}
$$

In [211]:
@defop
def MaxPool1d(
    X: Annotated[torch.Tensor, Scoped[S1 | S2]],
    seq: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    k: int,
    kernel: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    seq2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    return reduce([kernel], Pool(X, seq, k, kernel, seq2), torch.max)

In [212]:
seq_size = 10

seq, seq2, kernel = map(name_to_symbol, ("seq", "seq2", "kernel"))

X = torch.randn(seq_size)[seq()]
MaxPool1d(X, seq, 2, kernel, seq2)

tensor([[ 0.3129],
        [ 1.0415],
        [-0.5359],
        [ 1.1539],
        [ 0.1567]])[seq2(), slice(None, None, None)]

In [213]:
@defop
def MaxPool2d(
    X: Annotated[torch.Tensor, Scoped[S1 | S2]],
    height: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    kh_size: int,
    kh: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    height2: Operation[[], torch.Tensor],
    width: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    kw_size: int,
    kw: Annotated[Operation[[], torch.Tensor], Scoped[S1 | S2]],
    width2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    y = Pool(Pool(X, height, kh_size, kh, height2), width, kw_size, kw, width2)
    return reduce([kh, kw], y, torch.max)

In [214]:
width_size = 9
height_size = 4

width, width2, height, height2, kw, kh = map(
    name_to_symbol, ("width", "width2", "height", "height2", "kw", "kh")
)

X = torch.randn(width_size, height_size)[width(), height()]
MaxPool2d(X, height, 2, kh, height2, width, 3, kw, width2)

tensor([[[2.2239],
         [1.4813],
         [1.2894]],

        [[1.7128],
         [1.7628],
         [1.4359]]])[height2(), width2(), slice(None, None, None)]

### Normalization Layers

$$
\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{standardize}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}ax}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}ax}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{standardize}}}(X) &= \frac{X - \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{mean}}}(X)}{\sqrt{\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{var}}}(X) + \epsilon}}
\end{aligned}
$$

In [215]:
@defop
def Mean(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    return reduce([ax], X, torch.sum) / sizesof(X)[ax]


@defop
def Mean2(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    ax2: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    sizes = sizesof(X)
    return reduce([ax, ax2], X, torch.sum) / (sizes[ax] * sizes[ax2])


@defop
def Variance(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    return Mean((X - Mean(X, ax)) ** 2, ax)


@defop
def Variance2(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    ax2: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
) -> torch.Tensor:
    return Mean2((X - Mean2(X, ax, ax2)) ** 2, ax, ax2)


@defop
def Standardize(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    new_ax: Operation[[], torch.Tensor],
) -> torch.Tensor:
    y = subst(X, {ax: new_ax()})
    return (y - Mean(X, ax)) / (Variance(X, ax) + torch.finfo(X.dtype).eps).sqrt()


@defop
def Standardize2(
    X: Annotated[torch.Tensor, Scoped[S1]],
    ax: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    ax2: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    new_ax: Operation[[], torch.Tensor],
    new_ax2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    y = subst(X, {ax: new_ax(), ax2: new_ax2()})
    return (y - Mean2(X, ax, ax2)) / (
        Variance2(X, ax, ax2) + torch.finfo(X.dtype).eps
    ).sqrt()

$$
\begin{aligned}
\text{BatchNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}batch},\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}} \\
\text{InstanceNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}} \\
\text{LayerNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer},\mathsf{\vphantom{fg}chans}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans},\mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

In [216]:
@defop
def BatchNorm(
    X: Annotated[torch.Tensor, Scoped[S1]],
    gamma: torch.Tensor,
    beta: torch.Tensor,
    batch: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    layer: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    batch2: Operation[[], torch.Tensor],
    layer2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    return Standardize2(X, batch, layer, batch2, layer2) * gamma + beta


@defop
def InstanceNorm(
    X: Annotated[torch.Tensor, Scoped[S1]],
    gamma: torch.Tensor,
    beta: torch.Tensor,
    layer: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    layer2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    return Standardize(X, layer, layer2) * gamma + beta


# same as BatchNorm
@defop
def LayerNorm(
    X: Annotated[torch.Tensor, Scoped[S1]],
    gamma: torch.Tensor,
    beta: torch.Tensor,
    chans: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    layer: Annotated[Operation[[], torch.Tensor], Scoped[S1]],
    chans2: Operation[[], torch.Tensor],
    layer2: Operation[[], torch.Tensor],
) -> torch.Tensor:
    return Standardize2(X, chans, layer, chans2, layer2) * gamma + beta

In [217]:
batch_size, chans_size, layer_size = 4, 3, 5
batch, batch2, chans, layer, layer2 = map(
    name_to_symbol, ("batch", "batch2", "chans", "layer", "layer2")
)

x = torch.randn(batch_size, chans_size, layer_size)[batch(), chans(), layer()]
g = torch.randn(chans_size)[chans()]
b = torch.randn(chans_size)[chans()]

BatchNorm(x, g, b, batch, layer, batch2, layer2)

tensor([[[ 0.2670, -0.1588, -1.2522, -0.7949, -0.4869],
         [ 0.3384, -0.1322, -0.8064,  0.9302,  0.0463],
         [ 2.1462,  2.1542,  1.9729,  2.6483,  2.8954]],

        [[ 0.4752, -0.9735, -0.6327, -2.0224, -1.0046],
         [-0.9306, -0.2433, -0.2080,  0.3491, -0.0462],
         [ 2.0179,  3.4996,  2.2253,  2.7595,  1.7771]],

        [[-2.1675,  0.3962, -0.9786,  1.1038,  0.6551],
         [-0.3270, -0.1066, -0.1575,  0.0266,  0.5415],
         [ 1.9313,  3.4777,  2.7468,  2.4394,  3.2106]],

        [[ 2.2192, -0.8806, -1.7958, -0.9050, -1.3091],
         [-0.3796, -0.4029, -0.8666, -0.0087,  0.6328],
         [ 2.8615,  2.2634,  2.4476,  2.4643,  2.6424]]])[batch2(), chans(), layer2()]

$$
\begin{aligned}
\text{GroupNorm}_k(X; \gamma, \beta) &= \left[ \mathop{\underset{\substack{\mathsf{\vphantom{fg}kernel},\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}chans}, \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X \right]_{(\mathsf{\vphantom{fg}chans},\mathsf{\vphantom{fg}kernel})\rightarrow \mathsf{\vphantom{fg}chans}} \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta \\
\end{aligned}
$$

$$
\begin{aligned}
|\mathsf{\vphantom{fg}kernel}| &= k\\
\gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}}.
\end{aligned}
$$

### Transformer

$$
\begin{aligned}
  I &\in \{0, 1\}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}vocab}} & \sum\limits_{\substack{\mathsf{\vphantom{fg}vocab}}} I &= 1 \\
  W &= (E \mathbin{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\odot}} I)\sqrt{|\mathsf{\vphantom{fg}layer}|} & E &\in \mathbb{R}^{\mathsf{\vphantom{fg}vocab}\times \mathsf{\vphantom{fg}layer}} \\
  P &\in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} \\
  P_{\mathsf{\vphantom{fg}seq}(p), \mathsf{\vphantom{fg}layer}(i)} &= \begin{cases}
    \sin((p-1) / 10000^{(i-1) / |\mathsf{\vphantom{fg}layer}|}) & \text{$i$ odd} \\ 
    \cos((p-1) / 10000^{(i-2) / |\mathsf{\vphantom{fg}layer}|}) & \text{$i$ even.}
  \end{cases}
\end{aligned}
$$

$$
\begin{aligned}
X^0 &= W+P \\
T^1 &= \text{LayerNorm}^1(\text{SelfAtt}^1(X^0)) + X^0\\
X^1 &= \text{LayerNorm}^{1^\prime}(\text{FFN}^1(T^1)) + T^1\\
&\vdotswithin{=} \\
T^{L} &= \text{LayerNorm}^L(\text{SelfAtt}^L(X^{L-1})) + X^{L-1}\\
X^{L} &= \text{LayerNorm}^{L^\prime}(\text{FFN}^L(T^L)) + T^L\\
O &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{softmax}}}(E \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X^L)
\end{aligned}
$$

$$
\begin{aligned}
  \text{LayerNorm}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}layer}} \\
  \text{LayerNorm}^l(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{XNorm}}}(X; \beta^l, \gamma^l).
\end{aligned}
$$

$$
\begin{aligned}
  \text{SelfAtt}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} \\
  \text{SelfAtt}^l(X) &= Y
\end{aligned}
$$

$$
\begin{aligned}
  |\mathsf{\vphantom{fg}seq}| &= |\mathsf{\vphantom{fg}seq2}| \\
  |\mathsf{\vphantom{fg}key}| = |\mathsf{\vphantom{fg}val}| &= |\mathsf{\vphantom{fg}layer}|/|\mathsf{\vphantom{fg}heads}| \\
  Q &= W^{l,Q} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X_{\mathsf{\vphantom{fg}seq}\rightarrow\mathsf{\vphantom{fg}seq2}} & W^{l,Q} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}key}} \\
  K &= W^{l,K} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X & W^{l,K} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}key}} \\
  V &= W^{l,V} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X & W^{l,V} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}val}} \\
  M & \in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}seq2}} \\
  M_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}seq2}(j)} &= \begin{cases}
    0 & i \leq j\\
    -\infty & \text{otherwise}
  \end{cases} \\
  Y &= W^{l,O} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}heads}\\ \mathsf{\vphantom{fg}val}}}{\vphantom{fg}\odot}} \text{Attention}(Q, K, V, M)_{\mathsf{\vphantom{fg}seq2}\rightarrow\mathsf{\vphantom{fg}seq}} & W^{l,O} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}val}\times \mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

$$
\begin{aligned}
  \text{FFN}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}layer}} \\
  \text{FFN}^l(X) &= X^2
\end{aligned}
$$

$$
\begin{aligned}
  X^1 &= \text{relu}(W^{l,1} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X + b^{l,1}) & W^{l,1} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}layer}} & b^{l,1} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
  X^2 &= \text{relu}(W^{l,2} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} X^1 + b^{l,2}) & W^{l,2} &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}hidden}} & b^{l,2} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}}.
\end{aligned}
$$

### LeNet

$$
\begin{aligned}
X^0 &\in \mathbb{R}^{\mathsf{\vphantom{fg}batch}\times \mathsf{\vphantom{fg}chans}[c_0] \times \mathsf{\vphantom{fg}height}\times \mathsf{\vphantom{fg}width}} \\
T^1 &= \text{relu}(\text{Conv}^1(X^0)) \\
X^1 &= \text{MaxPool}^1(T^1) \\
T^2 &= \text{relu}(\text{Conv}^2(X^1)) \\
X^2 &= \text{MaxPool}^2(T^2)_{(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width},\mathsf{\vphantom{fg}chans})\rightarrow\mathsf{\vphantom{fg}layer}} \\
X^3 &= \text{relu}(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}layer}} & b^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
O &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}classes}}}{\vphantom{fg}\mathrm{softmax}}} (W^4 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} X^3 + b^4) & W^4 &\in \mathbb{R}^{\mathsf{\vphantom{fg}classes}\times \mathsf{\vphantom{fg}hidden}} & b^4 &\in \mathbb{R}^{\mathsf{\vphantom{fg}classes}}\end{aligned}
$$

$$
\begin{aligned}
X^2 &= \text{MaxPool}^2(T^2) \\
X^3 &= \text{relu}(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}\\ \mathsf{\vphantom{fg}width}\\ \mathsf{\vphantom{fg}chans}}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}height}\times \mathsf{\vphantom{fg}width}\times \mathsf{\vphantom{fg}chans}}.
\end{aligned}
$$

$$
\begin{aligned}
\text{Conv}^l(X) &= \text{Conv2d}(X; W^l, b^l)_{\mathsf{\vphantom{fg}chans2}\rightarrow\mathsf{\vphantom{fg}chans}}
\end{aligned}
$$

$$
\begin{aligned}
W^l & \in \mathbb{R}^{\mathsf{\vphantom{fg}chans2}[c_l] \times \mathsf{\vphantom{fg}chans}[c_{l-1}] \times \mathsf{\vphantom{fg}kh}[kh_l] \times \mathsf{\vphantom{fg}kw}[kw_l]} \\
b^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans2}[c_l]}
\end{aligned}
$$

$$
\begin{aligned}
\text{MaxPool}^l(X) &amp;= \text{MaxPool2d}_{ph^l,ph^l}(X).
\end{aligned}
$$

In [218]:
@defop
def Relu(X: torch.Tensor) -> torch.Tensor:
    return torch.maximum(X, torch.tensor(0))

In [219]:
Relu(x)

tensor([[[0.0000, 0.0000, 0.7641, 0.3760, 0.1145],
         [0.4051, 0.0000, 0.0000, 1.4258, 0.0000],
         [1.1718, 1.1479, 1.6898, 0.0000, 0.0000]],

        [[0.0000, 0.5275, 0.2382, 1.4178, 0.5540],
         [0.0000, 0.0000, 0.0000, 0.4236, 0.0000],
         [1.5552, 0.0000, 0.9354, 0.0000, 2.2747]],

        [[1.5410, 0.0000, 0.5318, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.7555],
         [1.8139, 0.0000, 0.0000, 0.2960, 0.0000]],

        [[0.0000, 0.4487, 1.2255, 0.4694, 0.8124],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.9130],
         [0.0000, 0.8218, 0.2714, 0.2214, 0.0000]]])[batch(), chans(), layer()]

In [220]:
(
    chans_size,
    kh_size,
    kw_size,
    hidden_size,
    height_size,
    width_size,
    classes_size,
    batch_size,
) = (3, 3, 4, 3, 14, 15, 5, 4)
(
    chans,
    chans2,
    kh,
    kw,
    height,
    height2,
    height3,
    width,
    width2,
    width3,
    hidden,
    classes,
    classes2,
    batch,
) = map(
    name_to_symbol,
    (
        "chans",
        "chans2",
        "kh",
        "kw",
        "height",
        "height2",
        "height3",
        "width",
        "width2",
        "width3",
        "hidden",
        "classes",
        "classes2",
        "batch",
    ),
)

W1 = torch.randn(chans_size, kh_size, kw_size, chans_size)[
    chans(), kh(), kw(), chans2()
]
b1 = torch.randn(chans_size)[chans2()]
W3 = torch.randn(hidden_size, 4, 4, chans_size)[hidden(), height3(), width3(), chans2()]
b3 = torch.randn(hidden_size)[hidden()]
W4 = torch.randn(hidden_size, classes_size)[hidden(), classes()]
b4 = torch.randn(classes_size)[classes()]
X0 = torch.randn(batch_size, chans_size, height_size, width_size)[
    batch(), chans(), height(), width()
]

T1 = Relu(
    Conv2d(X0, W1, b1, chans, kh_size, kh, height, height2, kw_size, kw, width, width2)
)
X1 = MaxPool2d(T1, height2, 3, kh, height3, width2, 3, kw, width3)
X3 = reduce([height3, width3, chans2], W3 * X1, torch.sum) + b3
O_ = Softmax(reduce([hidden], W4 * X3, torch.sum) + b4, classes, classes2)
O_

tensor([[[0.0000e+00],
         [0.0000e+00],
         [0.0000e+00],
         [0.0000e+00]],

        [[0.0000e+00],
         [0.0000e+00],
         [0.0000e+00],
         [0.0000e+00]],

        [[1.5932e-07],
         [1.0565e-36],
         [8.7251e-37],
         [0.0000e+00]],

        [[1.0000e+00],
         [1.0000e+00],
         [1.0000e+00],
         [1.0000e+00]],

        [[0.0000e+00],
         [0.0000e+00],
         [0.0000e+00],
         [0.0000e+00]]])[classes2(), batch(), slice(None, None, None)]